In [ ]:
import torch
from torch import nn
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import MultiStepLR
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from transformers import AutoTokenizer
from utils.data_utils import AG_NEWS_DATASET
from utils.constants import *
from model.transformer import Transformer as Transformer_origin
from utils.training import Learner

from quantization.fully_quantize import Model
from quantization.pytorch_api import ModelQuant

%load_ext autoreload
%autoreload 2

In [ ]:
# load dataset
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
train_dl, test_dl = AG_NEWS_DATASET(tokenizer).load_data()

# create model
model = Model(4,
                tokenizer.vocab_size,
                BASELINE_MODEL_NUMBER_OF_LAYERS,
                BASELINE_MODEL_NUMBER_OF_HEADS,
                BASELINE_MODEL_DIM)

# model = quantizer(model, 8, True)

# model = ModelQuant(4,
#                 tokenizer.vocab_size,
#                 BASELINE_MODEL_NUMBER_OF_LAYERS,
#                 BASELINE_MODEL_NUMBER_OF_HEADS,
#                 BASELINE_MODEL_DIMENSION)

model.model_name = 'transformer'

# loss func
loss_fn = nn.CrossEntropyLoss()

# simple optimizer
optim = Adam(model.parameters(), lr= 1e-4)
scheduler = MultiStepLR(optim, milestones=[10,15], gamma=0.1)

train_config ={'model': model,
               'loss_fn': loss_fn,
               'optim': optim,
               'datasets': [train_dl, test_dl],
               'epochs': 10,
               'batch_size': BATCH_SIZE,
               'scheduler': scheduler,
               'exp_name': "quant_all",
               }

# training
learner_ag_news = Learner(train_config)

In [ ]:
learner_ag_news.train()